In [3]:
# | echo: false
# | output: false

import sys
import os

sys.path.append(os.path.abspath("../../"))
sys.path.append(os.path.abspath("./social-media-handle-checker"))

%reload_ext autoreload
%autoreload 2

import asyncio
from typing import Any, Literal
from python_utils.get_browser import get_browser_page_async
from app import check_social_media_handle

## Instagram Account

In [ ]:
await check_social_media_handle("instagram", "test")

In [ ]:
tasks = [
    async_availability_status(resolve_instagram_username(username, lambda x, y: None)) 
    for username 
    in ['we', 'us', 'instagram']
]
await asyncio.gather(*tasks)

## LinkedIn User Account

In [ ]:
await async_availability_status(resolve_linkedin_username("USERNAME", "in"))

## LinkedIn Company Account

In [ ]:
await async_availability_status(resolve_linkedin_username("USERNAME", "company"))

In [ ]:




def resolve_username_from_title(username: str):
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto(uri, wait_until='load')
        await page.wait_for_timeout(300)
        title: str = await page.title()
        await close()
        return username.lower() in title.lower()
    return resolve

def resolve_document_contains(search: str, evaluate_str: str) -> bool:
    async def resolve(uri: str) -> bool:
        page, close = await get_browser_page_async()
        await page.goto(uri)
        inner_text = await page.evaluate(evaluate_str)
        await close()
        return search.lower() in inner_text.lower()
    return resolve

class UsernameChecker:
    def __init__(self, username):
        self.username = username
        self.platforms: dict[Literal["name", "uri", "resolve", "message"], Any]= [
            # {
            #     "name": "LinkedIn Company Page",
            #     # can replace "www." with "de.", ".ke", ".ug", etc
            #     "uri": f"https://www.linkedin.com/company/{username}",
            #     "resolve": resolve_linkedin_username,
            # },
            # {
            #     "name": "LinkedIn User Account",
            #     # can replace "www." with "de.", ".ke", ".ug", etc
                  #     # # linkedin private user => kamau
            #     "uri": f"https://www.linkedin.com/in/{username}",
            #     "resolve": resolve_linkedin_username,
            #     # https://github.com/tomquirk/linkedin-api
            #     "message": (
            #         "❌ Taken or Private."
            #         " 😶 LinkedIn private user profiles are not easy to check,"
            #         " login into LinkedIn and go to"
            #         f' "https://www.linkedin.com/in/{username}"')
            # },
            {
                "name": "Instagram Account",
                "resolve": resolve_instagram_username(username),
            },
            # {
            #     "name": "Threads Account",
            #     # trailing / is important here
            #     "uri": f"https://www.instagram.com/{username}/",
            #     "resolve": resolve_instagram_username(username),
            # },
            # {
            #     "name": "Youtube Channel",
            #     "uri": f"https://www.youtube.com/@{username}",
            #     "resolve": resolve_document_contains(f"@{username}", "document.body.outerHTML"),
            # },
            # {
            #     "name": "Tiktok Account",
            #     "uri": f"https://www.tiktok.com/@{username}",
            #     "resolve": resolve_username_from_title(username),
            # },
            # {
            #     "name": "X/Twitter Account",
            #     "uri": f"https://x.com/{username}",
            #     # not accurate!
            #     # https://github.com/sherlock-project/sherlock/blob/master/sherlock_project/resources/data.json
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Pinterest Account",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Snapchat Account",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Facebook User Account",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Facebook Page",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Reddit User",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Reddit Subreddit",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # },
            # {
            #     "name": "Gmail Email",
            #     "uri": f"https://www.pinterest.com/{username}/",
            #     "resolve": resolve_document_contains("doesn't exist", "document.body.innerText"),
            # }
        ]

    async def check_async(self):
        tasks = [self._check_async(**platform) for platform in self.platforms]
        return await asyncio.gather(*tasks)




In [ ]:

# Create checker instance
checker = UsernameChecker("One_MorePlace")
# Check availability
await checker.check_async()

In [112]:
from requests_tor import RequestsTor

rt = RequestsTor(tor_ports=(9050,), tor_cport=9051)

# Make a GET request through the Tor network
response = rt.get('https://api.ipify.org/')

# Print the IP address obtained through Tor
print(response.text) 

192.42.116.211


In [113]:
response = rt.get('https://api.ipify.org/')

# Print the IP address obtained through Tor
print(response.text) 

192.42.116.211


In [133]:
import shutil
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(
headless = True,
proxy = { "server": "socks5://localhost:9050" },
executable_path = shutil.which("google-chrome") or shutil.which("chromium-browser") or shutil.which("chromium"))
context = await browser.new_context(
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0')
page = await browser.new_page()
page_response = await page.goto("http://httpbin.org/ip", referer="https://www.instagram.com/")
print((await page_response.body()).decode())

{
  "origin": "185.220.101.107"
}



In [114]:
import re

def get_json_value(page_source, key, value_pattern):
    pattern = rf'[\'"]?{key}[\'"]?\s*:\s*[\'"]?({value_pattern})[\'"]?'
    match = re.search(pattern, page_source, flags=re.IGNORECASE)
    return match.group(1) if match else None

profile_response = rt.get(
    "https://www.instagram.com/", 
    allow_redirects = False,
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
    }
    )

csrf_token = get_json_value(profile_response.text, "csrf_token", "[\w-]+") or ""
appId = get_json_value(profile_response.text, "appId", "\d+") or ""
client_revision = get_json_value(profile_response.text, "client_revision", "\d+") or ""
x_ig_app_id = get_json_value(profile_response.text, "X-IG-App-ID", "\d+")
               
csrf_token, appId, client_revision, x_ig_app_id

('LIEhmFKHlvz3RYhbJ2bzly', '936619743392459', '1019001996', '936619743392459')

In [115]:
profile_response.cookies.get_dict()

{'csrftoken': 'LIEhmFKHlvz3RYhbJ2bzly'}

In [116]:
response = rt.post(
    'https://www.instagram.com/api/v1/web/accounts/web_create_ajax/attempt/', 
    headers = {
        "Content-Type": 'application/x-www-form-urlencoded',
        "x-csrftoken": csrf_token,
        "x-ig-app-id": x_ig_app_id,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0',
        # # not very much important
        # "X-Requested-With": "XMLHttpRequest",
        # "X-Instagram-AJAX": client_revision,
        # "X-IG-App-ID": appId,
    }, 
    data = {
        'email': '',
        'failed_birthday_year_count': '{}',
        'first_name': '',
        'username': 'yellow',
        'opt_into_one_tap': 'false',
        'use_new_suggested_user_name': 'true'
    },
    
    # data = "email=&failed_birthday_year_count=%7B%7D&first_name=&username=saut11&opt_into_one_tap=false&use_new_suggested_user_name=true",
    cookies = profile_response.cookies
    )
response.text

'{"message":"","spam":true,"status":"fail"}'

In [118]:
response.request.body

('email=&failed_birthday_year_count=%7B%7D&first_name=&username=yellow&opt_into_one_tap=false&use_new_suggested_user_name=true',
 {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/x-www-form-urlencoded', 'x-csrftoken': 'LIEhmFKHlvz3RYhbJ2bzly', 'x-ig-app-id': '936619743392459', 'Cookie': 'csrftoken=LIEhmFKHlvz3RYhbJ2bzly', 'Content-Length': '124'})